In [2]:
import torch
import os
from torchvision.utils import save_image
from tqdm import tqdm
import torch.nn as nn

In [3]:
# --- Set device and load model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels=1, feature_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # latent_dim x 1 x 1 → 4x4
            nn.ConvTranspose2d(latent_dim, feature_g * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_g * 16),
            nn.ReLU(True),
        
            # 4x4 → 8x8
            nn.ConvTranspose2d(feature_g * 16, feature_g * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 8),
            nn.ReLU(True),
        
            # 8x8 → 16x16
            nn.ConvTranspose2d(feature_g * 8, feature_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),
        
            # 16x16 → 32x32
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),
        
            # 32x32 → 64x64
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
        
            # 64x64 → 128x128
            nn.ConvTranspose2d(feature_g, feature_g // 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g // 2),
            nn.ReLU(True),
        
            # 128x128 → 256x256
            nn.ConvTranspose2d(feature_g // 2, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )


    def forward(self, x):
        return self.gen(x)

In [5]:
latent_dim = 100
generator = Generator(latent_dim=latent_dim).to(device)
generator.load_state_dict(torch.load("generator.pth", map_location=device))
generator.eval()

Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNo

In [6]:
# --- Generate and save images ---
os.makedirs("generated_fakes", exist_ok=True)

batch_size = 32
num_images = 6380+6380
num_batches = (num_images + batch_size - 1) // batch_size

In [12]:
with torch.no_grad():
    for batch_idx in tqdm(range(num_batches), desc="Generating fakes"):
        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_imgs = generator(noise)

        for i, img in enumerate(fake_imgs):
            if batch_idx * batch_size + i >= num_images:
                break
            filename = f"generated_fakes/fake_{batch_idx * batch_size + i:04d}.png"
            save_image(img, filename, normalize=True)

print("✅ 1000 fake images saved in 'generated_fakes'")

Generating fakes: 100%|██████████████████████████████████████████████████████████████| 399/399 [14:10<00:00,  2.13s/it]

✅ 1000 fake images saved in 'generated_fakes'
